<a href="https://colab.research.google.com/github/maxla777/Project-2/blob/Monique/Monique/ML_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AdaBoost

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

In [2]:
# Usage from scikit learn documentation
# To update to fit our project
# from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_iris
# from sklearn.ensemble import AdaBoostClassifier

# X, y = load_iris(return_X_y=True)
# clf = AdaBoostClassifier(n_estimators=100)
# scores = cross_val_score(clf, X, y, cv=5)
# scores.mean()

Random Forest

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
# Fit a SKLearn linear regression using just the training set (X_train, y_train):
# model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
# model.fit(X_train, y_train)

# Make a prediction of "y" values from the X_test dataset
# predictions = model.predict(X_test)

# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
# Results = y_test.to_frame()
# Results["Predicted Value"] = predictions
# Results